## Lens injection in multiple DC2 cutout images

In this notebook, we will show how one can inject simulated lenses to multiple DC2 cutout images and create a catalog of strong lense.

For this, we should follow following steps:
1. Create a lens population using sim-pipeline
2. Generate random ra and dec using random_ra_dec function in RSP module
3. Then, finally one can should call multiple_lens_injection function in RSP module

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import sys
sys.path.insert(0, '/home/nkhadka/notebooks/mytutorials/sim-pipeline/')

## Generate Galaxy-Galaxy Lens Population

In [2]:
# define a cosmology
from astropy.cosmology import FlatLambdaCDM
cosmo = FlatLambdaCDM(H0=70, Om0=0.3)

# define a sky area
from astropy.units import Quantity
sky_area = Quantity(value=0.1, unit='deg2')


# define limits in the intrinsic deflector and source population (in addition to the skypy config file)
kwargs_deflector_cut = {'band': 'g', 'band_max':28, 'z_min': 0.01, 'z_max': 2.5}
kwargs_source_cut = {'band': 'g', 'band_max':28, 'z_min': 0.1, 'z_max': 5.}

# import GGLensPop class
from sim_pipeline.gg_lens_pop import GGLensPop
# run skypy pipeline and make galaxy-galaxy population class
gg_lens_pop = GGLensPop(lens_type='all-galaxies', source_type='galaxies', 
                        kwargs_deflector_cut=kwargs_deflector_cut, kwargs_source_cut=kwargs_source_cut,
                        kwargs_mass2light=None, skypy_config=None, sky_area=sky_area, cosmo=cosmo)

## Initiate Butler
Butler is necessary to access and process DC2 data. So, one needs to initiate it.

In [4]:
import lsst.daf.butler as dafButler
config = "dp02"
collection ='2.2i/runs/DP0.2'
butler = dafButler.Butler(config, collections=collection)
skymap = butler.get("skyMap")

## Import RSP module
This module can be used to do all the operation necessary for the lens injection in DC2 data.
We will choose 100 random ra, dec pair in the DC2 data and we inject random gg lenses in these ra, dec position. Injected_lens_catalog contains following images and information,
1. Lens to be injected in r-band.
2. Corresponding DC2 cutout image in r-band.
3. r, g, and i band DC2 cutout images with corresponding injected lenes.
4. Center of the DC2 cutout images where lenses have been injected.

In [6]:
from sim_pipeline import RSP

In [7]:
##provide minimum and maximum limits for ra and dec. Also, specify how many ra, dec pair you want.
ra, dec=RSP.random_ra_dec(55, 70, -43, -30, 100)

In [8]:
## specify pixel number and pixel scale of the images. Here we have chosen 64 and 0.2 respectively. 
##We should choose pixel scale = 0.2 to match the pixel scale scale of DC2 data.
injected_lens_catalog=RSP.multiple_lens_injection(gg_lens_pop, ra, dec, 64, 0.2, butler, flux=None)

In [9]:
injected_lens_catalog

lens,cutout_image,injected_lens_r,injected_lens_g,injected_lens_i,cutout_center
"float32[64,64]","float32[64,64]","float32[64,64]","float32[64,64]","float32[64,64]",float64[2]
7.212907e-05 .. 3.801158e-05,0.04526387 .. 0.0012604285,0.045336 .. 0.00129844,0.012054982 .. -0.02789364,-0.02833417 .. 0.037127506,57.75274808279288 .. -33.06445760574345
0.0 .. 0.0,0.054079402 .. -0.00026809695,0.054079402 .. -0.00026809695,-0.04303897 .. -0.0057039997,-0.025019845 .. 0.03534288,69.70483305012444 .. -30.88770895837744
0.0 .. 0.0,0.026953297 .. 0.022407167,0.026953297 .. 0.022407167,0.0066534695 .. -0.031524204,0.02840236 .. -0.117760226,66.17442177550421 .. -42.39907037288373
0.0 .. 0.0,-0.0334646 .. -0.043052245,-0.0334646 .. -0.043052245,0.008833369 .. -0.006079131,-0.04008431 .. -0.03130091,64.92146541322535 .. -33.2527141673532
7.9638325e-05 .. 5.2746385e-05,-0.012805905 .. 0.032482266,-0.012726267 .. 0.032535013,0.03236037 .. -0.016172385,-0.035030346 .. -0.050459325,65.59677720838707 .. -35.51604711590282
0.0 .. 0.0,0.014122078 .. 0.37310293,0.014122078 .. 0.37310293,0.012617236 .. 0.21369311,0.0510176 .. 0.48133945,68.06463179629851 .. -38.90265627541665
0.0023257434 .. 0.002961688,0.101967916 .. -0.0022828756,0.10429366 .. 0.00067881244,0.015178914 .. 0.030310284,0.23011267 .. 0.053892285,59.86562854893438 .. -30.028598619972037
0.0001430288 .. 3.6917627e-06,-0.098768175 .. -0.038902782,-0.098625146 .. -0.03889909,-0.017543046 .. -0.02733806,-0.08249867 .. -0.048880957,62.40245910971964 .. -35.20759429140013
0.0 .. 0.0,-0.030746682 .. -0.03608451,-0.030746682 .. -0.03608451,0.035569225 .. -0.023898318,-0.028907584 .. -0.06162432,64.35777634293943 .. -41.46579216378508
